In [1]:
import wrestlr

%load_ext wrestlr

In [2]:
from siuba.siu import _, strip_symbolic

_ >> _ + _

█─>>
├─_
└─█─+
  ├─_
  └─_

## Basic

### Atomic Types

In [3]:
%%wrestlr -p -m siuba

'a'    # Str

1      # Int

1.2    # Dbl

1i     # Complex (TODO)

TRUE   # Bool

'a'

1

1.2

1j

True


In [4]:
%%wrestlr -p

# TODO!
Inf

NaN

NA

NULL

inf

nan

<NA>

None


### Syntax

In [5]:
%%wrestlr -p

x <- 1

# TODO: map correctly to range for simple cases
1:3

mtcars$hp

# Note: treats like "[["
mtcars["hp"]

mtcars[["hp"]]

x = 1

range(1,3)

mtcars.hp

mtcars['hp']

mtcars['hp']


In [6]:
%%wrestlr -p -m siuba

a$b$.c

a$"b c"

"z"

a.b['.c']

a['b c']

'z'


In [7]:
%%wrestlr -p -m siuba

mtcars$hp$"z.a"

mtcars.hp['z.a']


### Names

Outside of NSE, periods are left untransformed right now, but can be handled in siuba.

Note that invalid python names are wrapped with backticks (eg \`x.bad\`). This breaks black formatting (an option in wrestlr), so I am thinking about alternatives!

In [8]:
%%wrestlr -p

x

.x

.

is.na

`x`

..NSE..(`x`)


x

`.x`

`.`

`is.na`

x

`..NSE..`(x)


### Names and getters

wrestlr treats `$` like property access, and `[[` like getting an item.

When it a name wouldn't work with `$`, like in the case of ``a$`a.bad name` ``, wrestlr switches to treating it like `[[`.

In [9]:
%%wrestlr -p

a$`x.bad`

a$`x.real bad`

a$`x_good`

# backticks in R are a name, not a string

a[[`x.bad`]]

a[[`x.real bad`]]

a[[`x_good`]]



a['x.bad']

a['x.real bad']

a.x_good

a[`x.bad`]

a[`x.real bad`]

a[x_good]


### Function calls

In [10]:
%%wrestlr -p

f(x)

f(x, na.rm = TRUE)

f(...)

f()

is.na(c(1))

c(1, 2, 3)

f(x)

f(x,na.rm = True)

f('...')

f()

`is.na`(c(1))

c(1,2,3)


### Pipes

In [11]:
%%wrestlr -p

mtcars %>% f(hp > 100)

mtcars %>% f(avg = mean(hp))

mtcars >> f(hp > 100)

mtcars >> f(avg = mean(hp))


In [12]:
%%wrestlr -p

mtcars %>% f(cyl) %>% f(avg = mean(hp))

mtcars >> f(cyl) >> f(avg = mean(hp))


In [13]:
%%wrestlr -p

result <- mtcars %>% f(hp > 100)

result = (mtcars >> f(hp > 100))


In [14]:
%%wrestlr -p
result <- mtcars %>% a() %>% b() %>% c()

result = (mtcars >> a() >> b() >> c())


## NSE verbs

### Escaping bare columns

In [15]:
%%wrestlr -p

f_standard(data, hp)

# Use ..NSE.. for testing
..NSE..(data, hp)

f_standard(data,hp)

`..NSE..`(data,_.hp)


In [16]:
%%wrestlr -p

# TODO: should not escape mtcars
filter(mtcars, hp > 100)

mtcars %>% filter(hp > 100)

mutate(avg = mean(hp))

mutate(avg = mean(hp / cyl))

mtcars %>% group_by(cyl) %>% summarize(avg = mean(hp))

filter(mtcars,_.hp > 100)

mtcars >> filter(_.hp > 100)

mutate(avg = mean(_.hp))

mutate(avg = mean(_.hp / _.cyl))

mtcars >> group_by(_.cyl) >> summarize(avg = mean(_.hp))


In [17]:
%%wrestlr -p
result <- mtcars %>%
    group_by(cyl) %>%
    summarize(avg = mean(hp))

result = (mtcars >> group_by(_.cyl) >> summarize(avg = mean(_.hp)))


In [18]:
from siuba.data import mtcars
from siuba import group_by, summarize, _
result = (mtcars >> group_by(_.cyl) >> summarize(avg = _.hp.mean()))

### Function calls to methods

By default, function calls are not displayed as methods, except for the test call, `NSE_method`.

In [19]:
%%wrestlr -p

mtcars %>% mutate(avg = mean(hp / cyl))

mtcars %>% mutate(avg = NSE_method(hp, cyl) + mpg)

mtcars >> mutate(avg = mean(_.hp / _.cyl))

mtcars >> mutate(avg = _.hp.NSE_method(_.cyl) + _.mpg)


You can calls to convert by configuring the Transformer.

In [20]:
from wrestlr.transformers import TransformExpr
METHODS = {
    "mean": "mean",
    "is.na": "isna"
}

custom = TransformExpr(method_calls = METHODS)

Here it is without the transformer.

In [21]:
%%wrestlr -p

mtcars %>% mutate(is.na(hp))

mtcars >> mutate(`is.na`(_.hp))


And here it is with!

In [22]:
%%wrestlr -p -t custom

mtcars %>% mutate(is.na(hp))

mtcars >> mutate(_.hp.isna())


## Executing

Code can be executed with the `-e` flag

In [26]:
from siuba.data import mtcars
from siuba import _, mutate, head

In [27]:
%%wrestlr -p -e
mtcars %>% mutate(res = hp + 2) %>% head()

mtcars >> mutate(res = _.hp + 2) >> head()


,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,res
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,112
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,112
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,95
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,112
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,177


You can specify a custom transformer, to handle converting function calls to methods.

In [31]:
from wrestlr.transformers import TransformExpr

my_transformer = TransformExpr(method_calls = {"mean": "mean"})

In [34]:
%%wrestlr -p -e -t my_transformer
mtcars %>% mutate(res = mean(hp) + 2) %>% head()

mtcars >> mutate(res = _.hp.mean() + 2) >> head()


,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,res
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,148.6875
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,148.6875
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,148.6875
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,148.6875
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,148.6875


Though, note that general handling of function arguments, like `na.rm=True` is not implemented.

In [35]:
%%wrestlr -p -t my_transformer
mtcars %>% mutate(res = mean(hp, na.rm = TRUE) + 2) %>% head()

mtcars >> mutate(res = _.hp.mean(na.rm = True) + 2) >> head()


One exception is for ggplot calls, which wrestlr attempts to reformat to the conventions used by `plotnine` (a ggplot port to python). It essentially works with two changes:

* `theme(axis.x.text=...)` uses underscores instead. Eg. `theme(axis_x_text = ...)`.
* `aes(factor(x), y)` must quote variables. Eg `aes("factor(x)", "y")`.